#### Prep genelists ###
- Prepare genelists, converting IDs to the version used (r6.28)

In [ ]:
#Imports
import sys
import os
import pandas as pd

sys.path.append('../scripts')

from annotation_utilities import *
from plot_helpers import *

%load_ext autoreload
%autoreload 2

In [ ]:
## Flybase ID problem explanation
from IPython.display import Image
Image('../../../resources/FlybaseID_problem.png')

In [ ]:
outdir = '../Figures/genesets'
os.makedirs(outdir, exist_ok=True)

In [ ]:
#Load all the genes possibly in the dataset and convert to version 6.32
result_file = os.path.join(results_dir, 'gene_quantification/summary_abundance_by_gene.csv')
result_genes = list(set(pd.read_csv(result_file, index_col=0).index))

In [ ]:
# Create the ID mapping table
id_dir = '/Users/mkthompson/Desktop/Davislab/3.4_NMJ_4Tu_4sU/3.4e_pipeline_dev/nmj_figures/resources/id_conversion/'
dmel628_file = os.path.join(id_dir, 'fbgn_annotation_ID_fb_2019_03.tsv')
dmel632_file = os.path.join(id_dir, 'fbgn_annotation_ID_fb_2020_01.tsv')
df = update_ids(result_genes, dmel628_file, dmel632_file)
df2 = resolve_splits(df)
#This will print the ones with no new ID
#For 6.28 -> 6.32, it is only 'FBgn0261846', which Flybase tags as a withdrawn gene model
#df2[pd.isnull(df2['new_ID'])]

In [ ]:
# Mark which genes are TFs, using the gene table from dmel6.32
res_dir = '/Users/mkthompson/Desktop/Davislab/3.10_brain_ss/brain_figures/resources/'
tf_file = os.path.join(res_dir, 'TFs_dmel632.txt')
tf_ids = set(pd.read_csv(tf_file, sep='\t', header=None)[0].tolist())
df2['TF'] = df2['new_ID'].isin(tf_ids)
TF_IDs_converted = df2[df2['TF']].reset_index()['index'].to_csv(os.path.join(outdir, 'all_TFs.csv'), header=False, index=False)

In [ ]:
# Parse the TFs provided from FlyTF.org
# https://www.mrc-lmb.cam.ac.uk/genomes/FlyTF/old_index.html
import pandas as pd
infile = '../../../resources/curated_genelists/all_candidates.csv'
tf_df = pd.read_csv(infile, sep='\t')
tf_df['verdict_TF'].unique()

In [ ]:
#https://www.mrc-lmb.cam.ac.uk/genomes/FlyTF/old_index.html
maybe_tfs = pd.read_html('https://www.mrc-lmb.cam.ac.uk/genomes/FlyTF/DNA_binding_proven_TF_maybe.html')[0]
yes_tfs = pd.read_html('https://www.mrc-lmb.cam.ac.uk/genomes/FlyTF/quite_sure_TFs.html')[0]

In [ ]:
len(yes_tfs)

In [ ]:
tf_df['verdict_TF'].value_counts()

In [ ]:
tf_df['verdict_DNA_BD'].value_counts()
#The 753 seems to be equal to YES + maybe. 
#Not sure if the 454 well supported candidates are annotated in this set though.

In [ ]:
tf_df.head()

In [ ]:
465+288

In [ ]:
#Flybase does not have an RNA-binding protein gene class. Where should I get this from then? GO annotation?